In [6]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [8]:
lemmatizer = WordNetLemmatizer

In [15]:
conversation = json.loads(open("conversation.json").read())

In [25]:
words = []
classes = []
documents = []
ignore_letters = [".",",","?","!"]

In [28]:
for con in conversation["conversation"]:
    for pat in con["patterns"]:
        word_list = nltk.word_tokenize(pat)
        words.append(word_list)
        documents.append((word_list, con["tag"]))
        if con['tag'] not in classes:
            classes.append(con['tag'])

In [29]:
print(documents)

[(['Hello'], 'greetings'), (['Hi'], 'greetings'), (['Hey'], 'greetings'), (['How', 'are', 'you', 'doing'], 'greetings'), (['What', "'s", 'up'], 'greetings'), (['What', 'is', 'the', 'weather'], 'weather'), (['How', 'is', 'it', 'outside'], 'weather'), (['Goodbye'], 'greeting'), (['See', 'you'], 'greeting')]


In [38]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

TypeError: lemmatize() missing 1 required positional argument: 'word'

In [39]:
print(words)

[['Hello'], ['Hi'], ['Hey'], ['How', 'are', 'you', 'doing'], ['What', "'s", 'up'], ['What', 'is', 'the', 'weather'], ['How', 'is', 'it', 'outside'], ['Goodbye'], ['See', 'you']]


In [40]:
classes = sorted(set(classes))

In [ ]:
pickle.dump(words, open("words.pkl","wb"))
pickle.dump(words, open("classes.pkl","wb"))

In [ ]:
training = []
output_empty = [0] * len(classes)

In [41]:
# end 17:00 source: https://www.youtube.com/watch?v=1lwddP0KUEg 